##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Train sentiment analysis models with TensorFlow Lite Model Maker

In this step, we will use the [Stanford Sentiment Treebank v2](https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf)(SST-2) dataset to train the model. The dataset contains more than 11,000 sentences from movie reviews and the sentiment (positive or negative) of each sentence. Here is some examples from the dataset:

| Sentence | Sentiment |
|-|-|
| This is a film well worth seeing , talking and singing heads and all . | Positive |
| What really surprises about Wisegirls is its low-key quality and genuine tenderness . | Positive |
| Though everything might be literate and smart , it never took off and always seemed static . | Negative |

We will use TensorFlow Lite Model Maker to train **text classification** models with this dataset. **Sentiment analysis** is one type of **text classification** because its goal is to classify a body of text into positive or negative sentiment. We will train two models:
* An `average word vector` model: This is a light-weight model which is less 1MB and also runs very fast.
* A `MobileBERT` model: This is a more accurate but larger model.

You also can choose a model architecture in advance and train only one model. However, it is recommended to train multiple models with different architecture and examine the trade-off between model size and accuracy and see which model is more suitable for your use case.

*Note: Before you proceed, make sure that the Colab notebook is in GPU mode to speed up model training.*

### Install the required packages

Let's start by installing TFLite Model Maker and import the required packages.

In [2]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 591 kB 12.8 MB/s 
     |████████████████████████████████| 1.1 MB 37.4 MB/s 
     |████████████████████████████████| 1.1 MB 44.9 MB/s 
     |████████████████████████████████| 213 kB 49.2 MB/s 
     |████████████████████████████████| 1.2 MB 42.7 MB/s 
     |████████████████████████████████| 64 kB 3.1 MB/s 
     |████████████████████████████████| 6.3 MB 43.5 MB/s 
     |████████████████████████████████| 120 kB 44.0 MB/s 
     |████████████████████████████████| 596 kB 50.6 MB/s 
     |████████████████████████████████| 103 kB 51.5 MB/s 
     |████████████████████████████████| 87 kB 6.7 MB/s 
     |████████████████████████████████| 840 kB 44.9 MB/s 
     |████████████████████████████████| 352 kB 53.8 MB/s 
     |████████████████████████████████| 47.6 MB 33 kB/s 
     |████████████████████████████████| 99 kB 9.2 MB/s 
     |████████████████████████████████| 208 kB 44.7 MB/s 


In [3]:
import numpy as np
import os

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')

### Download the SST-2 dataset

In [4]:
data_dir = tf.keras.utils.get_file(
      fname='SST-2.zip',
      origin='https://dl.fbaipublicfiles.com/glue/data/SST-2.zip',
      extract=True)
data_dir = os.path.join(os.path.dirname(data_dir), 'SST-2')

7454720/7439277 [==============================] - 1s 0us/step


## Train our first model: an `Average Word Vector` model

This is the light-weight model architecture that can be trained in a few minutes. The model is small and runs fast with decent accuracy.

### Specify the model architecture

In [5]:
awv_spec = model_spec.get('average_word_vec')

### Pre-process data
Convert the SST-2 dataset to input format that is required by TFLite Model Maker. Please note that there are two portion of our dataset:
* `train.tsv`: The **training** dataset that the model will learn from.
* `dev.tsv`: The **evaluation** dataset that the model doesn't see when it is trained. This is used to test if the sentiment analysis is able to generalize well on new data that it has never seen before.

In [6]:
awv_train_data = DataLoader.from_csv(
      filename=os.path.join(os.path.join(data_dir, 'train.tsv')),
      text_column='sentence',
      label_column='label',
      model_spec=awv_spec,
      delimiter='\t',
      is_training=True)
awv_test_data = DataLoader.from_csv(
      filename=os.path.join(os.path.join(data_dir, 'dev.tsv')),
      text_column='sentence',
      label_column='label',
      model_spec=awv_spec,
      delimiter='\t',
      is_training=False)

INFO:tensorflow:Saved vocabulary in /tmp/tmprn7l5mzz/e72d242a17446c5dc91aa41e181ce914_vocab.


### Train the model
We use TFLite Model Maker to train a TensorFlow model with the training dataset.

In [7]:
awv_model = text_classifier.create(awv_train_data, model_spec=awv_spec, epochs=10)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/10
2104/2104 [==============================] - 16s 6ms/step - loss: 0.6736 - accuracy: 0.5739
Epoch 2/10
2104/2104 [==============================] - 13s 6ms/step - loss: 0.5348 - accuracy: 0.7436
Epoch 3/10
2104/2104 [==============================] - 13s 6ms/step - loss: 0.4290 - accuracy: 0.8081
Epoch 4/10
2104/2104 [==============================] - 13s 6ms/step - loss: 0.3862 - accuracy: 0.8301
Epoch 5/10
2104/2104 [==============================] - 13s 6ms/step - loss: 0.3637 - accuracy: 0.8422
Epoch 6/10
2104/2104 [==============================] - 13s 6ms/step - loss: 0.3493 - accuracy: 0.8515
Epoch 7/10
2104/2104 [==============================] - 13s 6ms/step - loss: 0.3383 - accuracy: 0.8564
Epoch 8/10
2104/2104 [==============================] - 13s 6ms/step - loss: 0.3304 - accuracy: 0.8621
Epoch 9/10
2104/2104 [==============================] - 13s 6ms/step - loss: 0.3236 - accuracy: 0.8644
Epoch 10/10
2104/2104 [==============================] - 13s 6ms/step - l

### Evaluate the model

Let's see how the model performs against new data in the evaluation dataset that it has never seen before.

In [8]:
awv_model.evaluate(awv_test_data)

28/28 [==============================] - 0s 5ms/step - loss: 0.5097 - accuracy: 0.8222


[0.5097113251686096, 0.8222476840019226]

### Export the model into TensorFlow Lite format

In [9]:
awv_model.export(export_dir='average_word_vec/')

INFO:tensorflow:Assets written to: /tmp/tmp9t1vrw9o/assets


INFO:tensorflow:Assets written to: /tmp/tmp9t1vrw9o/assets


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Saved vocabulary in /tmp/tmprz3a704h/vocab.txt.


...

INFO:tensorflow:The associated file that has been been packed to the model is:


INFO:tensorflow:['labels.txt', 'vocab.txt']


INFO:tensorflow:['labels.txt', 'vocab.txt']


INFO:tensorflow:TensorFlow Lite model exported successfully: average_word_vec/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: average_word_vec/model.tflite


## Optional: Train our second model: an `MobileBERT` model

`MobileBERT` models are more accurate than `Average Word Vector` models but they are also larger. It will take about 30 minutes to be train our `MobileBERT` sentiment analysis model.

### Specify the model architecture

You will see that all steps below are mostly the same as training a `Average Word Vector` sentiment analysis model. Here we use `MobileBert` model specification instead of the `Average Word Vector` specification.

In [10]:
mb_spec = model_spec.get('mobilebert_classifier')

### Pre-process data
Convert the SST-2 dataset to input format that is required by TFLite Model Maker. As `MobileBERT` models and `Average Word Vector` models take different input format, we'll need to run the data preprocess step again.

In [11]:
mb_train_data = DataLoader.from_csv(
      filename=os.path.join(os.path.join(data_dir, 'train.tsv')),
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      delimiter='\t',
      is_training=True)
mb_test_data = DataLoader.from_csv(
      filename=os.path.join(os.path.join(data_dir, 'dev.tsv')),
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      delimiter='\t',
      is_training=False)

### Train the model
We use TFLite Model Maker to train a TensorFlow model with the training dataset.

In [12]:
mb_model = text_classifier.create(mb_train_data, model_spec=mb_spec, epochs=3)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/3
1403/1403 [==============================] - 1441s 983ms/step - loss: 0.4869 - test_accuracy: 0.8503
Epoch 2/3
1403/1403 [==============================] - 1372s 978ms/step - loss: 0.1302 - test_accuracy: 0.9539
Epoch 3/3
1403/1403 [==============================] - 1382s 985ms/step - loss: 0.0770 - test_accuracy: 0.9759


### Evaluate the model

Let's see how the model performs against new data in the evaluation dataset that it has never seen before.

In [13]:
mb_model.evaluate(mb_test_data)

28/28 [==============================] - 11s 230ms/step - loss: 0.3997 - test_accuracy: 0.9014


[0.39973363280296326, 0.9013761281967163]

### Export the model into TensorFlow Lite format

In [14]:
mb_model.export(export_dir='mobilebert/')

INFO:tensorflow:Assets written to: /tmp/tmp6xp3x6xk/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp6xp3x6xk/saved_model/assets


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Saved vocabulary in /tmp/tmpcis582oj/vocab.txt.


...

INFO:tensorflow:The associated file that has been been packed to the model is:


INFO:tensorflow:['labels.txt', 'vocab.txt']


INFO:tensorflow:['labels.txt', 'vocab.txt']


INFO:tensorflow:TensorFlow Lite model exported successfully: mobilebert/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: mobilebert/model.tflite


## Optional: Compare the two models

Let's see the model size:

In [15]:
print('Average Word Vector')
!ls -all --block-size=KB average_word_vec/*.tflite
print('MobileBERT')
!ls -all --block-size=KB mobilebert/*.tflite

Average Word Vector
-rw-r--r-- 1 root root 776kB Oct 31 03:27 average_word_vec/model.tflite
MobileBERT
-rw-r--r-- 1 root root 25820kB Oct 31 04:40 mobilebert/model.tflite


You can optionally use the [TFLite benchmark tool](https://www.tensorflow.org/lite/performance/measurement#benchmark_tools) to measure the each model speed on mobile phones. The model speed of a certain model architecture does not change no matter which dataset you use to train the model. Therefore, you can refer to the model speed data on TensorFlow Lite website without running the benchmark by yourself.

Let's compare our two sentiment analysis models:

| Model Architecture | Accuracy | Size | Speed |
|-|-|-|-|
| Average Word Vector | 83% | 0.77MB | 0.05ms |
| MobileBERT | 90% | 25.7MB | 83ms |

*Note: Model speed was measured on Pixel 4 CPU, running on 2 threads.*

## Download the TFLite model.

We want to start small with a light-weight model, so the `Average Word Vector` model seems to be more suitable for our use case. Let's download it.

In [16]:
!cp average_word_vec/model.tflite sentiment_analysis.tflite

from google.colab import files
files.download('sentiment_analysis.tflite') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now you have finished training the two TFLite sentiment analysis models. Let's go back to the next step in the codelab.